In [66]:
import numpy as np
from collections import deque
from math import *
from itertools import permutations
from random import randint

import copy
import graphviz as gv
import networkx as nx
import pygraphviz as pgv
from networkx.drawing.nx_agraph import graphviz_layout
import tempfile
from PIL import Image

import numpy as np
from pathlib import Path
import imageio as io

Eps = '_'
MaxLength = 0

greedy_path_color = '#FF0000'
cycle_color = '#0000FF'
OPT_color = '#FF0000'
greedy_build_color = '#0000FF'
normal_edge_color = '#444444'
falling_color = '#00FF00'

In [67]:
def overlap(s1, s2):
    for i in reversed(range(1, min(len(s1), len(s2)) + 1)):
        if s1[-i:] == s2[:i]:
            return i
    return 0

def prefix(s1, s2):
    return len(s1) - overlap(s1, s2)

def suffix(s1, s2):
    return len(s2) - overlap(s1, s2)

def leng(st):
    return 0 if st == Eps else len(st)

def make_all_substrings_list(strings):
    all_substrings_set = set()
    for i_str in strings:
        for l in range(len(i_str)+1):
            for r in range(l, len(i_str)+1):
                all_substrings_set.add(i_str[l:r])
    all_substrings = sorted(list(all_substrings_set), key=lambda x: (len(x), x))
    all_substrings[0] = Eps
    return all_substrings

def make_pref_suff_list(strings):
    strings_set = set()
    for i_str in strings:
        for i in range(len(i_str)):
            strings_set.add(i_str[:i])
            strings_set.add(i_str[i:])
    strings_output = sorted(list(strings_set), key=lambda x: (len(x), x))
    strings_output[0] = Eps
    return strings_output

In [70]:

def edges_out(E_all, v):
    return [e for e in E_all if e[0] == v]
def edges_in(E_all, v):
    return [e for e in E_all if e[1] == v]
def edges_down_out(E_all, v):
    return [e for e in E_all if e[0] == v and leng(e[1]) < leng(v)]
def edges_down_in(E_all, v):
    return [e for e in E_all if e[1] == v and leng(e[0]) < leng(v)]
def ballance(E, v):
    return sum([0 if e[0]==e[1] else 1 if e[0]==v else -1 for e in E if v in e ])
output = '-'#*(sum(len(s) for s in input_strings)+1)
answer = ''
Hint = 'Hint'
OPT_string = 'abbaacdbbabc'
exist = []
cycles = []
def weak_component(V, E, v, marked = False):
    ans = [v]
    que = deque()
    que.append(v)
    E1 = [copy.copy(e) for e in E]
    if marked:
        E1 = [copy.copy(E[i]) for i in range(len(E)) if exist[i]] 
    while len(que) > 0:
        u = que.pop()
        for e in edges_out(E1, u):
            if e[1] in V and not e[1] in ans:
                ans.append(e[1])
                que.appendleft(e[1])
        for e in edges_in(E1, u):
            if e[0] in V and not e[0] in ans:
                ans.append(e[0])
                que.appendleft(e[0])
    return ans

def is_eulerian(V, E):
    if len(V) == len(weak_component(V, E, V[0])):
        imbal_in = []
        imbal_out = []
        for v in V:
            dif = ballance(E, v)
            if dif > 0:
                imbal_in.append(v)
            elif dif < 0:
                imbal_out.append(v)
            if not (len(imbal_in) < 2 and len(imbal_out) < 2):
                return False
        if len(imbal_in) != len(imbal_out):
            return False
        elif len(imbal_in) == 1:
            return ballance(E, imbal_in[0])*ballance(E, imbal_out[0]) == -1
        else:
            return True
    else:
        return False
    
    
HG=pgv.AGraph(directed=True, strict=False)
def make_hierarchical_graph(strings):
    global HG
    HG=pgv.AGraph(directed=True, strict=False)
    
    substrings = make_all_substrings_list(strings)
    
    HG.add_nodes_from(substrings)
    HG.add_node(output)
    #HG.add_node(OPT_string)
    #HG.add_node(Hint)
    for length in range(MaxLength+1)[::-1]:
        subgraph = pgv.AGraph()
        if length == 0:
            subgraph.graph_attr['rank']='source' 
        elif length == MaxLength:
            subgraph.graph_attr['rank']='sink' 
        else:
            subgraph.graph_attr['rank']='same' 
        
        #if length == MaxLength:
        #    subgraph.add_node(OPT_string)
            
        for string in substrings:
            if len(string) == length and string != Eps:
                subgraph.add_node(string)
        #if length == MaxLength:
        if length == 0:
            subgraph.add_node(output)
            #subgraph.add_node(Hint)
            
            subgraph.add_node(Eps)
            
            
        tmp = HG.add_subgraph(subgraph, rank='same')
    for v1 in HG.nodes():
        for v2 in HG.nodes():
            if (v1 != v2):
                if(v1 == Eps and len(v2) == 1) or (v2 == Eps and len(v1) == 1) or (v1[1:] == v2) or (v1 == v2[:-1]):
                    HG.add_edge(v1, v2, color = 'black')
    
    for st in substrings:
        if st in strings:
            HG.get_node(st).attr['shape']='square'
            HG.get_node(st).attr['color']='black'
        else:
            HG.get_node(st).attr['shape']='egg'
            HG.get_node(st).attr['color']='black'
            
    return(HG)

filename = 'img/graph'
images = []
do = True
filenumber = 0
exist = []
V = []
mrk = []
E = []
path = []
OPT = []
fall_edges = []

Hint_colors = [greedy_build_color,
               greedy_path_color,
               OPT_color,
               cycle_color,
               falling_color,
               falling_color]
Hints = ['Build Greedy\'s edges',
         'Find eulerian cycle - Greedy path',
         'Build path of OPT SCS',
         'Cycle cover',
         'Merge cycle cover and OPT path',
         'Falling of edges']
Hint_stage = 0
color_only_fall = False
picture = True
def frame(v = '#$%', color = 'goldenrod2'):
    global HG, filenumber, path, V, picture
    if picture:
        E1 = [list(e) for e in E]
        for i in range(len(path)):
            path[i] = Eps if path[i]=='' else path[i]
        for i in range(len(E1)):
            E1[i][0] = Eps if E1[i][0]=='' else E1[i][0]
            E1[i][1] = Eps if E1[i][1]=='' else E1[i][1]
        path = [[path[iv], path[iv + 1]] for iv in range(len(path) - 1)]
        for u in HG.nodes():
            HG.get_node(u).attr['color'] = 'black'
            HG.get_node(u).attr['style'] = 'solid'
        HG.get_node(output).attr['color'] = 'white'
        HG.get_node(output).attr['label'] = answer if answer != '' else 'GREEDY - 3'
        #HG.get_node(OPT_string).attr['color'] = 'white'
        #HG.get_node(OPT_string).attr['fontcolor'] = OPT_color
        #HG.get_node(Hint).attr['fontcolor'] = Hint_colors[Hint_stage]
        #HG.get_node(Hint).attr['color'] = 'white'
        #HG.get_node(Hint).attr['label'] = Hints[Hint_stage]
        #HG.get_node(Hint).attr['pos'] = str(float(HG.get_node(Eps).attr['pos'].split(',')[0])/2)+','+HG.get_node(Eps).attr['pos'].split(',')[1]
        if v != '#$%':
            HG.get_node(v).attr['color'] = color
            HG.get_node(v).attr['style'] = 'filled'

        for e in HG.edges():
            HG.get_edge(e[0], e[1]).attr['color'] = normal_edge_color
        for e in path:
            if HG.get_edge(e[0], e[1]).attr['color'] != greedy_path_color:
                HG.get_edge(e[0], e[1]).attr['color'] = greedy_path_color
            else:
                HG.get_edge(e[0], e[1]).attr['color'] += ":"+greedy_path_color
            E1.remove(e)
        if not color_only_fall:
            for e in cycles:
                HG.get_edge(e[0], e[1]).attr['color'] += ":"+cycle_color
            #for e in OPT:
            #    HG.get_edge(e[0], e[1]).attr['color'] += ":"+OPT_color
            for e in E1:
                HG.get_edge(e[0], e[1]).attr['color'] += ":"+greedy_build_color
        for e in fall_edges:
            HG.get_edge(e[0], e[1]).attr['color'] += ":"+falling_color


        HG.draw(filename+str(filenumber)+'.png')
        images.append(Image.open(filename+str(filenumber)+'.png'))
    filenumber+=1

def pause(steps = 3, v = '#$%', color = 'goldenrod2'):
    return 0
    for i in range(steps):
        frame(v, color)
    
def add_edge(e, extra = False):
    if not e in E or extra:
        E.append(e)
    i0 = V.index(e[0])
    i1 = V.index(e[1])
    if mrk[i0] == 0:
        mrk[i0] = mrk[i1]
    elif mrk[i1] == 0:
        mrk[i1] = mrk[i0]
    else:
        col = int(mrk[i0])
        for i in range(len(mrk)):
            if mrk[i] == col:
                mrk[i] = mrk[i1]
def path_summ(a, path):
    s = 0
    for i in range(len(path)-1):
        s += a[path[i]][path[i+1]]
    return s

def max_ATSP(a):
    n = len(a)
    ans_val = 0
    ans_path = []
    for per in permutations(range(n)):
        cur_val = 0
        for v in range(n - 1):
            cur_val += a[per[v]][per[v+1]]
        if(ans_val < cur_val):
            ans_val = cur_val
            ans_path = per
    return ans_path

def make_overlap_graph(strings):
    n = len(strings)
    a = np.zeros([n, n])
    for i in range(n):
        for j in range(n):
            str1 = strings[i]
            str2 = strings[j]
            overlap = 0
            for l in range(min(len(str1), len(str2)) + 1):
                if(str1.endswith(str2[:l])):
                    overlap = l
            a[i][j] = overlap
    return a

def print_superstring(strings):
    print('\n---BEGIN PRINTING SCS---')
    a = make_overlap_graph(strings)
    path = max_ATSP(a)
    lenght = len(strings[path[0]])
    print(strings[path[0]],end='')
    for i in range(1, len(path)):
        lenght += len(strings[path[i]][int(a[path[i-1]][path[i]]):])
        print(strings[path[i]][int(a[path[i-1]][path[i]]):],end='')
    print()
    shift = 0
    print(strings[path[0]])
    for i in range(1, len(path)):
        shift += len(strings[path[i-1]]) - a[path[i-1]][path[i]]
        print(' '*int(shift), end = '')
        print(strings[path[i]])
        
    print('Length =',lenght)
    print('Compression =',path_summ(a, path))
    
    print('----END PRINTING SCS----\n')
    return True

def make_optimal_cycle(strings):
    a = make_overlap_graph(strings)
    n = len(strings)
    if n <= 1:
        return []
    ans_val = 0
    ans_path = []
    path = []
    for per in permutations(range(n)):
        cur_val = 0
        for v in range(n):
            cur_val += a[per[v]][per[(v+1)%n]]
        if(ans_val <= cur_val):
            ans_val = cur_val
            ans_path = per
    for iistr in range(len(ans_path)):
        str1 = strings[ans_path[iistr]]
        str2 = strings[ans_path[(iistr + 1)%n]]
        over = str2[:overlap(str1,str2)]
        path = path + [str1[i:] for i in range(len(str1) - len(over))] + [str2[:i] for i in range(len(over), len(str2))]
    path.append(path[0])
    i_mi = 0
    for i in range(len(path)):
        if len(path[i])<len(path[i_mi]):
            i_mi = i
    if len(path[i_mi])!=0:
        addy = [path[i_mi][i:] for i in range(len(path[i_mi])+1)] + [path[i_mi][:i] for i in range(1,len(path[i_mi]))]
        path = path[:i_mi]+addy+path[i_mi:]
    for ip in range(len(path)):
        if path[ip] =='':
            path[ip] = Eps
    return [[path[i],path[i+1]] for i in range(len(path)-1)]

In [71]:
def exp(input_strings):
    global picture
    global output
    output = '-'*(sum(len(s) for s in input_strings)+1)
    global answer
    answer = ''
    global Hint#
    global OPT_string #
    global exist
    exist.clear()
    global cycles
    cycles.clear()
    global filename
    global images
    images.clear()
    global filenumber
    filenumber = 0
    global V
    V = make_all_substrings_list(input_strings)[::-1]
    global mrk
    global E
    E.clear()
    global path
    path.clear()
    global OPT
    OPT.clear()
    global fall_edges
    fall_edges.clear()
    global Hint_stage
    Hint_stage = 0
    global color_only_fall
    color_only_fall = False
    global Eps, MaxLength
    MaxLength = max(len(st) for st in input_strings)
    HG = make_hierarchical_graph(input_strings)
    #HG.draw('file.ps',prog='circo')
    HG.layout('dot')
    
    mrk = [0]*len(V)
    
    pause() #Start
    Vsorted = sorted(list(V), key=lambda x: [float(y) for y in HG.get_node(x).attr['pos'].split(',')][::-1])[::-1]
    V = Vsorted
    #Make first edges for Greedy from input strings
    for iv in range(len(input_strings)):
        mrk[V.index(input_strings[iv])] = iv+1
        add_edge(HG.out_edges(input_strings[iv])[0])
        add_edge(HG.in_edges(input_strings[iv])[0])
        #frame(input_strings[iv], greedy_build_color)
    #frame()
    ###Begin Greedy###
    
    #Make Greedy edges
    for v in Vsorted:
        iv = V.index(v)
        length = leng(v)
        ball = ballance(E, v)
        if ball > 0:
            for e in HG.in_edges(v):
                if leng(e[0]) < leng(v):
                    ball-=1
                    add_edge(e)
                    if ball == 0:
                        break
            while ball > 0:
                for e in HG.in_edges(v):
                    if leng(e[0]) < leng(v):
                        ball-=1
                        add_edge(e, True)
                        if ball == 0:
                            break

        elif ball < 0:
            for e in HG.out_edges(v):
                if leng(e[1]) < leng(v):
                    ball+=1
                    add_edge(e)
                    if ball == 0:
                        break
            while ball < 0:
                for e in HG.out_edges(v):
                    if leng(e[1]) < leng(v):
                        ball+=1
                        add_edge(e, True)
                        if ball == 0:
                            break
        iu = iv
        
        
        #Check connectivity
        if len([iw for iw in range(iu+1, len(mrk)) if mrk[iu] == mrk[iw] and leng(V[iu]) >= leng(V[iw])]) == 0:
            if [mrk[iu] == mrk[iw] or 0 == mrk[iw] for iw in range(len(mrk))].count(False) != 0 and mrk[iu] != 0:
                if is_eulerian([V[iw] for iw in range(len(V)) if mrk[iw] == mrk[iu]], [list(e) for e in E]):
                    breaked = False
                    for e in HG.out_edges(V[iu]):
                        if not e in E and leng(e[1]) < leng(V[iu]):
                            add_edge(e)
                            breaked = True
                            break
                    if not breaked:
                        for e in HG.out_edges(V[iu]):
                            if leng(e[1]) < leng(V[iu]):
                                add_edge(e, True)
                                break
                    breaked = False
                    for e in HG.in_edges(V[iu]):
                        if not e in E and leng(e[0]) < leng(V[iu]):
                            add_edge(e)
                            breaked = True
                            break
                    if not breaked:
                        for e in HG.in_edges(V[iu]):
                            if leng(e[0]) < leng(V[iu]):
                                add_edge(e, True)
                                break

        #frame(V[iv], greedy_build_color)
    frame(Eps, greedy_build_color)
    
    i_mi = 0
    notsu = True
    for i in range(len(E)):
        if E[i][1]=='_':
            notsu = False
            break
        if len(E[i][1]) < len(E[i_mi][1]):
            i_mi = i
    if notsu:
        addy = []
        mimi = E[i_mi][1]
        for i in range(len(mimi)):
            addy.append((mimi[i:],mimi[i+1:]))
        for i in range(len(mimi)):
            addy.append((mimi[:i],mimi[:i+1]))
        E = E[:i_mi]+addy+E[i_mi:]
    
    #End making Greedy edges
    #Hint_stage += 1
    ##Begin find Greedy path
    #V_set = set()
    #for e in E:
    #    V_set.add(e[0])
    #V = list(V_set)
    #path = [V[0]]
    #out_len = len(output)
    #out_i = 0
    #for v in V:
    #    if(leng(v) < leng(path[0])):
    #        path[0] = v
    #exist = [True]*len(E)
    #while exist.count(True) > 0:
    #    for ie in range(len(E)):
    #        e = E[ie]
    #        v = path[-1]
    #        if e[0] == v and exist[ie]:
    #            exist[ie] = False
    #            VV = [w for w in V if len([iw for iw in range(len(E)) if w in E[iw] and exist[iw]])]
    #            if exist.count(True) == 0 or len(VV) == len(weak_component(VV,[list(E[ie]) for ie in range(len(E)) if exist[ie]],VV[0])):
    #                path.append(e[1])
    #                if leng(e[0]) < leng(e[1]):
    #                    answer +=path[-1][-1]
    #                break
    #            else:
    #                exist[ie] = True
#
    #    #frame(path[-1], greedy_path_color)
    ##pause()
    ####End Greedy###
    
    Hint_stage += 1
    ###Make optimal string###
    OPT1 = make_optimal_cycle(input_strings)
    for e in OPT1:
        OPT.append(e)
        #frame(e[1], OPT_color)
    pause()
    Hint_stage += 1
    OPT_len = len(OPT1)
    Greedy_len = len(E)
    frame()
    
    ####Begin cycle cover
    #cycles = []
    #for v in input_strings:
    #    cycles.append(HG.out_edges(v)[0])
    #    cycles.append(HG.in_edges(v)[0])
    #    #frame(v, cycle_color)
#
#
    #V = Vsorted
    #for v in Vsorted:
    #    length = leng(v)
    #    ball = ballance(cycles, v)
    #    if ball > 0:
    #        for e in HG.in_edges(v):
    #            if leng(e[0]) < leng(v):
    #                ball-=1
    #                cycles.append(e)
    #                if ball == 0:
    #                    break
    #        while ball > 0:
    #            for e in HG.in_edges(v):
    #                if leng(e[0]) < leng(v):
    #                    ball-=1
    #                    cycles.append(e)
    #                    if ball == 0:
    #                        break
#
    #    elif ball < 0:
    #        for e in HG.out_edges(v):
    #            if leng(e[1]) < leng(v):
    #                ball+=1
    #                cycles.append(e)
    #                if ball == 0:
    #                    break
    #        while ball < 0:
    #            for e in HG.out_edges(v):
    #                if leng(e[1]) < leng(v):
    #                    ball+=1
    #                    cycles.append(e)
    #                    if ball == 0:
    #                        break
    #    #frame(v, cycle_color)
    ##pause()
    #Hint_stage += 1
    #
    #fall_edges_tmp = OPT + OPT
    #fall_edges = [[e[0], e[1]] for e in fall_edges_tmp]
    #color_only_fall = True
    #pause()
#
    #Hint_stage += 1
#
    ##print(fall_edges)
    #for v in Vsorted:
    #    #frame(v, falling_color)
    #    while len(edges_down_out(fall_edges, v))*len(edges_down_in(fall_edges, v)) > 0:
    #        e_l = edges_down_in(fall_edges, v)[0]
    #        e_r = edges_down_out(fall_edges, v)[0]
    #        v_l = e_l[0]
    #        v_r = e_r[1]
    #        v_d = v_l[1:]
    #        if v_d == '':
    #            v_d = Eps
    #        if v in input_strings and len(edges_out(fall_edges, v))*len(edges_in(fall_edges, v)) == 1:
    #            #frame(v, falling_color)
    #            break
    #        elif len(edges_out(fall_edges, v))*len(edges_in(fall_edges, v)) == 1:
    #            if len(edges_down_out(fall_edges, v))*len(edges_down_in(fall_edges, v)) == 1:
    #                if v_l == v_r:
    #                    fall_edges.remove(e_l)
    #                    fall_edges.remove(e_r)
    #                else:
    #                    fall_edges.remove(e_l)
    #                    fall_edges.append([v_l, v_d])
    #                    fall_edges.remove(e_r)
    #                    fall_edges.append([v_d, v_r])
    #            else:
    #                #frame(v, falling_color)
    #                break
    #        else:
    #            tmp = [[e[0], e[1]] for e in fall_edges]
    #            tmp.remove(e_l)
    #            tmp.remove(e_r)
    #            if v_l != v_d:
    #                tmp.append([v_l, v_d])
    #                tmp.append([v_d, v_r])
    #            V_set = set()
    #            for e in tmp:
    #                V_set.add(e[0])
    #            if is_eulerian(list(V_set), tmp):
    #                fall_edges = [copy.copy(e) for e in tmp]
    #            else:
    #                #frame(v, falling_color)
    #                break
    #        #frame(v, falling_color)       
    #pause()
    #
    #
    #pathE = [[path[iv], path[iv + 1]] for iv in range(len(path) - 1)]
    #mark_fall = [False]*len(fall_edges)
    #mark_greed = [False]*len(pathE)
    #for i in range(len(pathE)):
    #    for j in range(len(mark_fall)):
    #        if fall_edges[j] == pathE[i] and not mark_fall[j]:
    #            mark_fall[j] = True
    #            mark_greed[i] = True
    #            break
    if OPT_len%2==1:
        print(OPT1)
    return [Greedy_len, OPT_len]#[mark_greed.count(False) == 0, OPT_len, Greedy_len, len(fall_edges)]
exp(['abab', 'abba', 'baba'])
#exp(['acca', 'acde', 'cbea', 'ccab', 'eacb'])
#exp(['aeae', 'cbea', 'cdcc', 'cddc', 'dccb', 'dcdc', 'eaea', 'edcd', 'eedb'])
1/0

ZeroDivisionError: division by zero

In [ ]:
def gen2(n,L, sigma):
    STR = ''
    rs = L
    N = int(n*rs*2/3)
    while True:
        ans = []
        stry1 = []
        for i in range(N):
            STR += chr(ord('a')+randint(0, sigma - 1))
        for i in range(n):
            st = ''
            l = int(i*rs/N)
            r = (l + rs)%N
            j = l
            while j != r:
                st += STR[j]
                j = (j+1)%N
            ans.append(st)
        goody = True
        for is1 in range(len(ans)):
            s1 = ans[is1]
            for is2 in range(len(ans)):
                s2 = ans[is2]
                if s2.find(s1) != -1 and is1 != is2:
                    goody = False
            if goody:
                stry1.append(s1)
        if not goody:
            continue
        else:
            break
    return sorted(ans)
def gen1(n,L,sigma):
    ans = []
    while len(ans) < n:
        st = ''
        for j in range(L):
            st += chr(ord('a')+randint(0, sigma - 1))
        if ans.count(st)==0:
            ans.append(st)
    return sorted(ans)
picture = False
#exp(['abbadd', 'bbdbbd', 'aaabca', 'bababc', 'cacaba', 'cbaacb'])
#exp(gen1(5,5,3))

In [ ]:
print([1,2,3][2:])
#images = []
#for fn in [filename+str(fi)+'.png' for fi in range(filenumber)]:
#    images.append(io.imread(fn))
#io.mimsave(filename+'.gif', images, duration = 0.2)
import sys
if sys.maxsize > 2**32:
    print('64-bit')
else:
    print('32-bit')

In [ ]:
import time
import itertools
start_time = time.clock()

def gen_all(s):
    ans = []
    for i1 in range(s):
        for i2 in range(s):
            for i3 in range(s):
                for i4 in range(s):
                    ans.append(chr(ord('a')+i1)+chr(ord('a')+i2)+chr(ord('a')+i3)+chr(ord('a')+i4))
    return ans
ally = []

def gen11(n):
    while True:
        ans = []
        while len(ans) < n:
            st = ally[randint(0, len(ally)-1)]
            if ans.count(st)==0:
                ans.append(st)
        let = set()
        for st in ans:
            let.update(set(st))
        if len(let) != s:
            continue
        return sorted(ans)

def gen21(n, s, k):
    L = ''
    while len(L) < 4*n*k:
        L+=chr(ord('a')+randint(0, s-1))
    ans = []
    L1 = L+L[:5]
    while len(ans) < n:
        pos = randint(0, len(L)-1)
        st = L1[pos:pos+4]
        if ans.count(st)==0:
            ans.append(st)
    return sorted(ans)
    
def gen22(n, s, k):
    while True:
        cont = 0
        L = ''
        while len(L) < 4*n*k:
            L+=chr(ord('a')+randint(0, s-1))
        ans = []
        L1 = L+L[:5]
        pos = 0
        while len(ans) < n:
            st = L1[pos:pos+4]
            if ans.count(st)==0:
                ans.append(st)
            pos= (pos + randint(1, 3))%len(L)
            cont+=1
            if cont > n**3:
                break
        let = set()
        for st in ans:
            let.update(set(st))
        if cont > n**3 or len(let) != s:
            continue
        return sorted(ans)
breaky = False
gr_op = []

In [ ]:
#['aaba', 'abaa', 'abbb', 'baab', 'babb', 'bbab', 'bbba'] 1/3
#['aadc', 'adca', 'bcdc', 'cabc', 'dcaa'] 1/3


#3 - 1.2857142857142858 	 ['abac', 'acab', 'acba', 'baca']
#4 - 1.1111111111111112 	 ['abcb', 'bbdc', 'bcab', 'cbca']
#4 - 1.1250000000000000 	 ['adba', 'babb', 'bbcd', 'bcdb', 'cdbd', 'cdcb', 'dbad', 'dcbd']
#4 - 1.1333333333333333 	 ['bbdb', 'bcdb', 'cabc', 'cdab', 'cdbc', 'dabb', 'dbca', 'dbdb']
#5 - 1.1538461538461537 	 ['aadb', 'bcbe', 'bcdb', 'cdbc', 'dbcd', 'ecdc']
#5 - 1.2500000000000000 	 ['aeed', 'cbda', 'dacb']
#6 - 1.0909090909090908 	 ['abcf', 'cfdd', 'daab', 'fefc']
#6 - 1.1111111111111112 	 ['bbfc', 'caff', 'fcaf', 'ffca']
#6 - 1.1176470588235294 	 ['accf', 'aeec', 'cefe', 'ecef', 'edcf', 'fbda', 'feae']
#6 - 1.1250000000000000 	 ['bbce', 'bfae', 'dafb', 'ebad', 'ebfe', 'febf']
#7 - 1.0909090909090908 	 ['agab', 'agag', 'bdag', 'ggba']
#7 - 1.1333333333333333 	 ['bfae', 'cbfc', 'cgdd', 'ddgg', 'dggc', 'eacg', 'fcbf', 'gddg']
#8 - 1.0769230769230769 	 ['aaec', 'aecb', 'bbaa', 'eagb', 'ecbb', 'gbda']
#8 - 1.1538461538461537 	 ['acbc', 'bacb', 'dbhe', 'fgdd', 'hedb']
#9 - 1.0952380952380953 	 ['aebb', 'bbfh', 'cgae', 'chch', 'chid', 'dihc', 'hchc', 'hidd', 'idfg']
#9 - 1.105263157894737 	 ['ahhg', 'ciga', 'ebeg', 'egea', 'fdci', 'gahh', 'hbeb', 'hfdc', 'hgah']
#10- 1.1428571428571428 	 ['afjg', 'dihe', 'febc', 'heia', 'jgaf']
#10- 1.0952380952380953 	 ['aidf', 'cfdh', 'dfai', 'ejjd', 'figg', 'geac', 'ggea', 'jjbe']
#3 - 1.2 	 ['bcbc', 'bccb', 'caca', 'cbcb']
#3 - 1.2 	 ['aacc', 'abca', 'acab', 'ccaa']
#4 - 1.3333333333333333 	 ['aadc', 'adca', 'bcdc', 'cabc', 'dcaa']
#5 - 1.1 	 ['bbdc', 'bdce', 'ccbb', 'cecc', 'daad', 'dcad', 'eddc', 'edec']
#6 - 1.0625 	 ['abcd', 'cdca', 'cffc', 'defe', 'eaed']
#7 - 1.0833333333333333 	 ['bfce', 'cefc', 'dgaf', 'fcea']
#8 - 1.0740740740740742 	 ['acbh', 'baae', 'bfgb', 'bhac', 'cdbf', 'dgcc', 'gegc', 'hadh', 'hgdb']
#9 - 1.0555555555555556 	 ['bceb', 'bice', 'cabc', 'dcfc', 'dhgf', 'edhg', 'fhed']


#for s in range(2, 13):
#    if breaky:
#        break
#    ally = gen_all(s)
#    if s==1:
#        for n in range(3,16):
#            if breaky:
#                break
#            start_time = time.time()
#            col1,col2,col3 = 0,0,0
#            for lis in itertools.combinations(ally,n):
#                #gg = gen41(n,s)
#                ans = exp(lis)
#                if not ans[0]:
#                    print(lis)
#                    breaky = True
#                    break
#                elif ans[1]<ans[2]:
#                    col3+=1
#                    gr_op.append(1.0*ans[2]/ans[1])
#                elif ans[1]!=ans[3]:
#                    col2+=1
#                elif ans[2]!=ans[3]:
#                    col1+=1
#                
#            print("Strings",n)
#            print("|Sigma|",s)
#            print("--- %s seconds ---" % (time.time() - start_time))
#            print(col1,col2,col3,'\n')
#    else:
#        for n in range(6,7):
#            if breaky:
#                break
#            all_time = time.time()
#            col1,col2,col3 = 0,0,0
#            for i in range(100):
#            #while (time.time() - all_time) < 500:
#                
#                lis = gen11(n)
#                ans = exp(lis)
#                if not ans[0]:
#                    print(lis)
#                    breaky = True
#                    break
#                elif ans[1]<ans[2]:
#                    col3+=1
#                    gr_op.append(1.0*ans[2]/ans[1])
#                    print(1.0*ans[2]/ans[1])
#                    if 1.0*ans[2]/ans[1] >= 1.15:
#                        print(lis)
#                elif ans[1]!=ans[3]:
#                    col2+=1
#                elif ans[2]!=ans[3]:
#                    col1+=1
#            print("Strings",n)
#            print("|Sigma|",s)
#            print(col1,col2,col3,'\n')

In [ ]:
#print(exp(['aaba', 'abaa', 'abbb', 'baab', 'babb', 'bbab', 'bbba']))
#print(exp(['aadc', 'adca', 'bcdc', 'cabc', 'dcaa']))
#print(exp(['abab', 'abba', 'baba']))
s, n = 9, 7
ally = gen_all(s)
all_time = time.time()
col1,col2,col3 = 0,0,0
#for n in range(3,10):
for i in range(35):
    #print(i)
    lis = gen11(n)#,s,66/100)#11(n)#
    #print(i, lis)
    ans = exp(lis)
    if ans[0] < 1:
        print(ans, lis)
        breaky = True
        break
    elif ans[0]/ans[1]>1:
        col3+=1
        print(n, s, ans[0]/ans[1], ans[0], ans[1],'\t',lis)
print('-----')
for i in range(35):
    #print(i)
    lis = gen22(n,s,66/100)#11(n)#
    #print(i, lis)
    ans = exp(lis)
    if ans[0] < 1:
        print(ans, lis)
        breaky = True
        break
    elif ans[0]/ans[1]>1:
        col2+=1
        print(n, s, ans[0]/ans[1], ans[0], ans[1],'\t',lis)
print(n, s)
print(col3, col2,'\n')
print("--- %s seconds ---" % (time.time() - all_time))
1/0

In [73]:
sorted(['eaea', 'cbae', 'cdcc', 'cddc', 'dccb', 'dcdc', 'aeae', 'adcd', 'aadb'])

['aadb', 'adcd', 'aeae', 'cbae', 'cdcc', 'cddc', 'dccb', 'dcdc', 'eaea']

In [ ]:
9 5 1.018181818181818 56 55 	 ['babb', 'bdce', 'cbeb', 'ccbe', 'daad', 'dcae', 'ddcd', 'ddce', 'ebec']

9 4 1.0833333333333333 52 48 	 ['acbb', 'cbac', 'cdcd', 'dbdc', 'dcbc', 'dccc', 'dcdc', 'ddaa', 'ddad']

9 3 1.0454545454545454 46 44 	 ['aaaa', 'abca', 'acbb', 'baac', 'bcab', 'caab', 'cabc', 'cbac', 'ccba']

9 2 1.0714285714285714 30 28 	 ['aaaa', 'aaab', 'aaba', 'abbb', 'baaa', 'baab', 'babb', 'bbab', 'bbba']
8 2 1.0769230769230769 28 26 	 ['aaaa', 'aaab', 'abab', 'abbb', 'baab', 'baba', 'bbaa', 'bbba']
7 2 1.1304347826086956 26 23 	 ['aaab', 'aaba', 'abaa', 'abbb', 'baaa', 'babb', 'bbbb']
6 2 1.1 22 20 	 ['aaab', 'abbb', 'baaa', 'babb', 'bbab', 'bbba']
5 2 1.0909090909090908 24 22 	 ['abab', 'abba', 'baab', 'baba', 'bbba']
3 2 1.1428571428571428 16 14 	 ['abab', 'abba', 'baba']
4 3 1.1111111111111112 20 18 	 ['aabb', 'aacb', 'baac', 'cbaa']
5 3 1.0714285714285714 30 28 	 ['aaac', 'acaa', 'baab', 'bacb', 'ccba']
6 3 1.0588235294117647 36 34 	 ['abbb', 'bacb', 'bcab', 'caab', 'cbac', 'ccab']
7 3 1.0666666666666667 32 30 	 ['acbb', 'babb', 'bbac', 'bbcb', 'bcba', 'bcca', 'cbbb']
8 3 1.1111111111111112 40 36 	 ['aaca', 'abab', 'abcc', 'acbc', 'bbca', 'cabc', 'cbca', 'ccab']
8 4 1.05 42 40 	 ['aadd', 'acbb', 'addc', 'caca', 'cacb', 'cccd', 'dcda', 'ddad']
6 4 1.0625 34 32 	 ['adbb', 'baac', 'ccdc', 'cdbc', 'dbcb', 'dccc']
5 4 1.1428571428571428 32 28 	 ['abcb', 'baab', 'bcaa', 'bdcd', 'cbba']
3 4 1.1111111111111112 20 18 	 ['baca', 'caba', 'dcbc']
6 5 1.0555555555555556 38 36 	 ['bcab', 'bdbd', 'ccea', 'cecb', 'eacc', 'ecdc']
7 5 1.05 42 40 	 ['adbe', 'bead', 'ccaa', 'dbae', 'ddcd', 'eabc', 'eacc']
5 6 1.0625 34 32 	 ['adfc', 'aeae', 'beca', 'cbad', 'dbca']
6 6 1.0555555555555556 38 36 	 ['abcf', 'acbf', 'aeee', 'affa', 'cfdd', 'ffaf']
7 6 1.0454545454545454 46 44 	 ['acdd', 'adff', 'bada', 'bbfb', 'daab', 'ecdd', 'fdbe']
8 6 1.0384615384615385 54 52 	 ['abda', 'acdb', 'adcb', 'bdab', 'ccce', 'deed', 'faea', 'fddc']
9 7 1.0384615384615385 54 52 	 ['acbf', 'afge', 'bgaf', 'dece', 'eaff', 'egde', 'ffea', 'gegf', 'ggbf']
8 7 1.04 52 50 	 ['aacg', 'acad', 'cfee', 'cggf', 'dace', 'dedg', 'eecf', 'gdcb']
7 7 1.0476190476190477 44 42 	 ['aaag', 'afda', 'bbfd', 'bcaf', 'dabc', 'eacb', 'ffdb']
9 8 1.0344827586206897 60 58 	 ['aaea', 'dfgd', 'dgfa', 'ehde', 'fadg', 'faff', 'gaec', 'habd', 'hbdh']
6 9 1.05 42 40 	 ['begc', 'daih', 'ehbi', 'gdha', 'hhfb', 'ihad']
6 8 1.05 42 40 	 ['bdgg', 'cbcd', 'cead', 'fghg', 'gedg', 'ggbd']



9 9 1.0416666666666667 50 48 	 ['adeh', 'bfdf', 'chfe', 'dbbf', 'ddbb', 'dfga', 'dfgd', 'gihc', 'hcdg']
9 9 1.0476190476190477 44 42 	 ['acbh', 'agfi', 'bgec', 'ddbg', 'deag', 'gghh', 'ghhd', 'hddb', 'ibge']
8 9 1.0232558139534884 44 43 	 ['befh', 'cdha', 'efha', 'fcef', 'fhfc', 'hagi', 'hehb', 'ifch']
8 9 1.0555555555555556 38 36 	 ['aaab', 'bffi', 'bgec', 'dbff', 'eaaa', 'figd', 'gdbf', 'ighe']
7 9 1.0555555555555556 38 36 	 ['bdfi', 'eahg', 'fidb', 'gcdb', 'hdah', 'hgea', 'idbh']
7 9 1.0666666666666667 32 30 	 ['aehf', 'aiga', 'cgdb', 'dbcg', 'ehfc', 'hfca', 'igad']
6 9 1.0625 34 32 	 ['bfgc', 'cgdh', 'fgcg', 'gcgb', 'iedi', 'ihag']
5 9 1.0344827586206897 30 29 	 ['cbeh', 'efac', 'ehbg', 'face', 'idfa']
6 8 1.0714285714285714 30 28 	 ['agdh', 'cdag', 'ehca', 'gdhc', 'gfbe', 'hcda']
6 8 1.0625 34 32 	 ['aghc', 'efdg', 'fdgh', 'ghee', 'hcbg', 'hhhf']
7 8 1.0909090909090908 36 33 	 ['acbc', 'agdg', 'bcbc', 'bcfa', 'bghe', 'facb', 'ghef']
8 8 1.0256410256410255 40 39 	 ['bcah', 'bfdc', 'cehb', 'ehbf', 'gcdc', 'ggcd', 'hehg', 'hghe']
8 8 1.0588235294117647 36 34 	 ['bafg', 'bebc', 'dbeb', 'ebcf', 'fdhc', 'fgfh', 'hcfd', 'hdbe']
9 8 1.0526315789473684 40 38 	 ['abga', 'bgad', 'bhdg', 'dgeb', 'efch', 'gebh', 'ghhc', 'hghh', 'hhca']
9 8 1.0526315789473684 40 38 	 ['adfa', 'ahee', 'cadf', 'cfah', 'dbdc', 'faha', 'gcad', 'habd', 'heea']
9 7 1.0526315789473684 40 38 	 ['affb', 'bacf', 'bbgd', 'bgdb', 'cefb', 'dbaf', 'fbac', 'fcca', 'ffbb']
9 7 1.0952380952380953 46 42 	 ['afdg', 'bbfa', 'cbbf', 'ccae', 'cegc', 'dgaf', 'fecb', 'fgbc', 'gbce']
8 7 1.0476190476190477 44 42 	 ['abfe', 'cabd', 'cbdf', 'cebc', 'ecbd', 'fdfg', 'feab', 'gece']
8 7 1.0526315789473684 40 38 	 ['agbd', 'ageg', 'beeb', 'cage', 'ccgb', 'ebdf', 'egca', 'gbdc']
6 7 1.0666666666666667 32 30 	 ['affb', 'ceaf', 'ddgd', 'fbdd', 'gddg', 'ggdc']
4 7 1.1 22 20 	 ['bdce', 'cegb', 'egbd', 'gfac']
5 6 1.0769230769230769 28 26 	 ['bfed', 'caee', 'deeb', 'ebee', 'eebf']
6 6 1.0666666666666667 32 30 	 ['aebf', 'bfee', 'daeb', 'dbfa', 'ecbd', 'fadb']
7 6 1.05 42 40 	 ['bebc', 'ccea', 'cdcf', 'dacc', 'eada', 'ecdb', 'feae']
8 6 1.105263157894737 42 38 	 ['bbbb', 'bbbf', 'bcfb', 'bfbd', 'ecce', 'eeaf', 'eeea', 'fbbc']
8 6 1.125 36 32 	 ['aabb', 'abbc', 'baed', 'bbca', 'bcaa', 'cdfe', 'ddbb', 'eddb']
9 6 1.0526315789473684 40 38 	 ['acbd', 'aebc', 'bccd', 'bfbd', 'ccbf', 'cdae', 'dafa', 'ebcc', 'facb']
4 5 1.1 22 20 	 ['baec', 'deee', 'ecde', 'eecd']
4 5 1.1111111111111112 20 18 	 ['aedb', 'bbdb', 'dbbc', 'edbb']
5 5 1.1538461538461537 30 26 	 ['acca', 'acde', 'cbea', 'ccab', 'eacb']
5 5 1.1111111111111112 20 18 	 ['aacb', 'acba', 'aced', 'baac', 'cbaa']
6 5 1.1428571428571428 32 28 	 ['bebe', 'beee', 'cccc', 'eade', 'ebeb', 'eeec']
6 5 1.0769230769230769 28 26 	 ['aaec', 'acde', 'aecc', 'bacd', 'ccea', 'deba']
7 5 1.1176470588235294 38 34 	 ['aedc', 'bbdd', 'bddd', 'cacb', 'cedd', 'dcae', 'dddd']
7 5 1.0625 34 32 	 ['aabc', 'cabd', 'ceed', 'dcee', 'edce', 'edda', 'eedc']
8 5 1.1176470588235294 38 34 	 ['abce', 'abde', 'babd', 'bceb', 'beca', 'ccab', 'ebcc', 'ecad']
8 5 1.0625 34 32 	 ['aecd', 'bbcc', 'bcca', 'ccda', 'cded', 'dabc', 'dbbc', 'ecde']
9 5 1.1176470588235294 38 34 	 ['baed', 'bbde', 'cade', 'ddbb', 'ddec', 'decd', 'ecad', 'ecdd', 'eddb']
9 5 1.1666666666666667 42 36 	 ['aeae', 'cbea', 'cdcc', 'cddc', 'dccb', 'dcdc', 'eaea', 'edcd', 'eedb']
8 4 1.0588235294117647 36 34 	 ['abdc', 'baab', 'cadb', 'cdda', 'dacb', 'dbaa', 'dcad', 'ddad']
8 4 1.0555555555555556 38 36 	 ['aada', 'adaa', 'baba', 'bddc', 'cacb', 'cbab', 'cdcc', 'daad']
7 4 1.0555555555555556 38 36 	 ['abbd', 'bbbd', 'bcbb', 'bdcc', 'cbbc', 'cdab', 'dbcc']
7 4 1.0625 34 32 	 ['cacb', 'cbcc', 'ccac', 'cccc', 'ccda', 'dacc', 'dbac']
6 4 1.0769230769230769 28 26 	 ['adbd', 'bbbd', 'bbdc', 'bdbb', 'dbdd', 'dcdb']
6 4 1.0833333333333333 26 24 	 ['adcd', 'badd', 'bdda', 'cdbd', 'dadc', 'ddad']
5 4 1.1 22 20 	 ['acbc', 'acdd', 'bcac', 'cacb', 'cdda']
5 4 1.0909090909090908 24 22 	 ['abcd', 'adbd', 'bdcd', 'cdad', 'dcda']
3 4 1.1111111111111112 20 18 	 ['badc', 'cacd', 'dcba']
9 4 1.0625 34 32 	 ['aaaa', 'aacd', 'bcdd', 'cdbc', 'cdcd', 'daac', 'dbcd', 'ddaa', 'ddda']
9 4 1.0810810810810811 40 37 	 ['abad', 'adcc', 'bada', 'bcbd', 'bdba', 'cbdb', 'ccdd', 'daba', 'dcaa']
3 3 1.1111111111111112 20 18 	 ['aabc', 'acca', 'bcaa']
3 3 1.1111111111111112 20 18 	 ['abbb', 'acba', 'baac']
4 3 1.1111111111111112 20 18 	 ['aaac', 'bccb', 'cbcc', 'ccbc']
4 3 1.125 18 16 	 ['abba', 'acab', 'babb', 'bbab']
5 3 1.0909090909090908 24 22 	 ['aaba', 'abaa', 'babb', 'babc', 'bbab']
5 3 1.1 22 20 	 ['abba', 'abcc', 'babc', 'bccb', 'cbab']
6 3 1.0909090909090908 24 22 	 ['abab', 'babc', 'bccb', 'cbab', 'cbcc', 'ccbc']
6 3 1.0833333333333333 26 24 	 ['abba', 'acca', 'bacb', 'bbaa', 'cabb', 'cbac']
7 3 1.0714285714285714 30 28 	 ['abba', 'abcb', 'acbc', 'babc', 'cacb', 'cbca', 'ccab']
7 3 1.0769230769230769 28 26 	 ['abcc', 'acab', 'acca', 'baca', 'bbac', 'bccb', 'cabb']
8 3 1.1538461538461537 30 26 	 ['accb', 'bbcc', 'bcac', 'bccc', 'cacc', 'cbab', 'cbca', 'ccbc']
8 3 1.125 36 32 	 ['aaca', 'acaa', 'accc', 'bacc', 'bbac', 'bcbb', 'cabc', 'cbac']
9 3 1.0909090909090908 36 33 	 ['aabc', 'abcc', 'acab', 'baca', 'bbcb', 'bcbb', 'cabc', 'cbac', 'ccbc']
9 3 1.0526315789473684 40 38 	 ['aaab', 'aabb', 'aacc', 'abab', 'abbc', 'baba', 'cbca', 'ccab', 'cccc']
9 2 1.0769230769230769 28 26 	 ['aaaa', 'aaab', 'aaba', 'abaa', 'abbb', 'baaa', 'bbab', 'bbba', 'bbbb']
7 2 1.0833333333333333 26 24 	 ['aaab', 'aaba', 'abaa', 'abbb', 'baaa', 'bbab', 'bbbb']
6 2 1.1 22 20 	 ['aaba', 'abaa', 'abba', 'baab', 'babb', 'bbab']
6 2 1.0909090909090908 24 22 	 ['abab', 'abba', 'baba', 'babb', 'bbab', 'bbbb']
5 2 1.0434782608695652 24 23 	 ['aaaa', 'aabb', 'baab', 'bbaa', 'bbbb']
4 2 1.125 18 16 	 ['abba', 'baab', 'babb', 'bbab']
4 2 1.1 22 20 	 ['abab', 'baaa', 'baab', 'baba']
3 2 1.1428571428571428 16 14 	 ['abab', 'abbb', 'baba']
